# Piping a Prompt, Model, and an Output Parser

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [2]:
pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [ ]:
# %load_ext dotenv
# %dotenv

In [4]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [5]:
%pip install -qU langchain-openai

In [6]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [7]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [8]:
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

So below ChatPromptTemplate is automatically defines as human prompt

In [9]:
chat_template = ChatPromptTemplate.from_messages([
    ('human',
     "I've recently adopted a {pet}. Could you suggest three {pet} names? \n" + list_instructions)])

In [10]:
print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [11]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [12]:
list_output_parser = CommaSeparatedListOutputParser()

So passing the value "dog" manually and checking the prompt

In [18]:
chat_template.invoke({'pet':'dog'})

ChatPromptValue(messages=[HumanMessage(content="I've recently adopted a dog. Could you suggest three dog names? \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`", additional_kwargs={}, response_metadata={})])

In [13]:
chat_template_result = chat_template.invoke({'pet':'dog'})

Below we can see we have passed to chatgpt then got three names

In [19]:
chat.invoke(chat_template_result)

AIMessage(content='Max, Bella, Cooper', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 50, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-Caxbq0WhmYfAiB3pLjvTDrIpJAHl5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--cf1443fe-d023-4f37-8856-07fda9a5dc8e-0', usage_metadata={'input_tokens': 50, 'output_tokens': 5, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
chat_result = chat.invoke(chat_template_result)

In [15]:
list_output_parser.invoke(chat_result)

['Max', 'Bella', 'Charlie']

The line chain = chat_template | chat | list_output_parser creates a LangChain Expression Language (LCEL) chain by combining a prompt template, a language model, and an output parser. Here's what each part does:

chat_template (ChatPromptTemplate): This is the prompt that defines the structure and content of the input to the language model. It takes a variable (in this case, {pet}) and inserts it into a predefined message, also including instructions for the output format from list_instructions.

chat (ChatOpenAI): This is the large language model (LLM) itself. It takes the structured prompt generated by chat_template as input and produces a response based on its training.

list_output_parser (CommaSeparatedListOutputParser): This component takes the raw output from the chat model and processes it. In this specific case, it expects a comma-separated string from the model and converts it into a Python list of strings.

The | operator pipes the output of one component as the input to the next. So, when you invoke chain.invoke({'pet':'dog'}):

The chat_template first formats the prompt with 'dog'.
The formatted prompt is then sent to the chat (GPT-4) model, which generates a comma-separated list of dog names.
Finally, list_output_parser takes that comma-separated string from the model and parses it into a clean Python list, making the output structured and easy to use programmatically.
This creates a clear, modular, and reusable sequence for processing user input through an LLM to get a structured output.

In [16]:
chain = chat_template | chat | list_output_parser

In [17]:
chain.invoke({'pet':'dog'})

['Max', 'Bella', 'Cooper']